# Glacier Gallery

In [ ]:
# https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
import panel as pn
import os, numpy as np, pandas as pd, cartopy.crs as ccrs, bokeh
import holoviews as hv, geoviews as gv, datashader as ds
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, Plot, Range1d, Text, TapTool, CustomJS, Legend, LegendItem
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_file, show, output_notebook, reset_output
from bokeh.models.markers import Circle
hv.extension('bokeh', width=100)

In [ ]:
# prepare coordinates
import math
from ast import literal_eval

def merc(Coords):
    lat = Coords[0]
    lon = Coords[1]

    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale    
    return (x, y)

## Load data

In [ ]:
df_all_all = pd.read_hdf('/home/zora/Dokumente/git/glacier-explorer/data/rgi62_era5_itmix_df.h5', 'df')
df_all_all['rgi_id']= list(df_all_all.axes[0])
df_all = df_all_all[['rgi_id', 'CenLon', 'CenLat', 'Area', 'Zmed','Lmax', 'Status','Connect', 'Form','TermType', 'Surging','Linkages', 'GlacierType','TerminusType', 'GlacierStatus', 'IsTidewater', 'IsNominal' ]].copy()
df_all.tail()

## Select glaciers and build lists with information about them

In [ ]:
# which one is columbia? : RGI60-01.10689 or RGI60-02.18415 

# Important: fill the list with increasing rgi-ids!
# RGI-ID, Name, location from where picture is loaded into app, Photo courtesy text, license abbrevation, license link, width image in hover, height image in hover, source link
Glacier_No = [('RGI60-01.01104', 'Lemon Creek Glacier', 
               'https://raw.githubusercontent.com/zschirmeister/glacier-gallery/dev3/glacier_photos/lemoncreek.jpg', 
               "Photo courtesy L. Bernier/National Snow and Ice Data Center",
               '', '',
               '390', '260',
              'http://nsidc.org/data/glacier_photo/search/image_info/lemoncreek20070917 '),
              
              ('RGI60-01.10689', 'Columbia Glacier',
               'https://upload.wikimedia.org/wikipedia/commons/7/7b/Columbia_Glacier_%28Alaska%29_by_Sentinel-2.jpg', 
               'Photo contains modified Copernicus Sentinel-2 data, (European Space Agency - ESA)  2018', 
               '(CC BY-SA 3.0 IGO)', 'https://creativecommons.org/licenses/by-sa/3.0/igo/deed.en',
               '300', '318',
              'https://commons.wikimedia.org/wiki/File:Columbia_Glacier_(Alaska)_by_Sentinel-2.jpg '),
             
              ('RGI60-03.04539', 'White Glacier', 
               'https://raw.githubusercontent.com/zschirmeister/glacier-gallery/dev3/glacier_photos/white.jpg',
               'Photo courtesy J. Alean/National Snow and Ice Data Center',
               '', '',
               '200', '133', # this is set as a very small picture, otherwise it would be outside the margin of the browser window
              'http://nsidc.org/data/glacier_photo/search/image_info/white20080702'),
              
              ('RGI60-05.11186', 'Mittivakkat Glacier', 
               'https://raw.githubusercontent.com/zschirmeister/glacier-gallery/dev3/glacier_photos/mittivakkat.jpg', 
               'Photo courtesy N. Knudsen/National Snow and Ice Data Center',
               '', '',
               '400', '300',
              'http://nsidc.org/data/glacier_photo/search/image_info/mittivakkat20130827?order=true '),
              
              ('RGI60-08.01097', 'Briksdalsbreen', 
               'https://raw.githubusercontent.com/zschirmeister/glacier-gallery/dev3/glacier_photos/briksdalsbreen.jpg',
               'Photo courtesy J. Perez Crespo/National Snow and Ice Data Center',
               '(CC BY-NC)', '',
               '259', '387',
              'http://nsidc.org/data/glacier_photo/search/image_info/briksdalsbreen20090730?order=true'),
              
              #('RGI60-09.00114', 'Severny Island Ice Cap', np.NaN, np.NaN, '', ''),
              #('RGI60-09.00471', 'Vavilov Ice Cap', np.NaN, np.NaN,  '', ''),
              ('RGI60-11.00897', 'Hintereisferner', 
               'https://raw.githubusercontent.com/zschirmeister/glacier-gallery/dev3/glacier_photos/hintereisferner.bmp', 
               'Photo courtesy A. Lambrecht/National Snow and Ice Data Center',
               '', '',
               '400', '265',
              'http://nsidc.org/data/glacier_photo/search/image_info/hintereisferner20060912?order=true'), 
              
              ('RGI60-11.01450', 'Großer Aletschgletscher', 
               'https://upload.wikimedia.org/wikipedia/commons/a/a1/Grosser_Aletschgletscher_3196.JPG', 
               'Photo courtesy Dirk Beyer',
               '(CC BY-SA 3.0)', 'https://creativecommons.org/licenses/by-sa/3.0/',
               '408', '306',
              'https://commons.wikimedia.org/wiki/File:Grosser_Aletschgletscher_3196.JPG'), 
              
              ('RGI60-12.00411', 'Tbilisa Glacier', 
               'https://raw.githubusercontent.com/zschirmeister/glacier-gallery/dev3/glacier_photos/tbilisa.jpg', 
               'Photo courtesy L. Tielidze/National Snow and Ice Data Center',
               '', '',
               '360', '272',
              'http://nsidc.org/data/glacier_photo/search/image_info/tbilisa20109999'),
              
              #('RGI60-13.04946', 'Samoilowich', np.NaN, np.NaN, '', ''), 
              ('RGI60-16.01638', 'Lewis', 
               'https://raw.githubusercontent.com/zschirmeister/glacier-gallery/dev3/glacier_photos/lewis.jpg', 
               'Photo courtesy R. Prinz/National Snow and Ice Data Center',
               '', '',
               '400', '300',
              'http://nsidc.org/data/glacier_photo/search/image_info/lewis20120223'),
              
              ('RGI60-16.02444', 'Artesonraju', 
               'https://upload.wikimedia.org/wikipedia/commons/0/06/Artesonraju_Peru.jpg', 
               'Photo courtesy Jonas Jancarik',
               '(CC BY 3.0)', 'https://creativecommons.org/licenses/by/3.0/deed.en',
               '400', '276',
              'https://commons.wikimedia.org/wiki/File:Artesonraju_Peru.jpg'), 
              
              ('RGI60-17.00312', 'Perito Moreno', 
               'https://raw.githubusercontent.com/zschirmeister/glacier-gallery/dev3/glacier_photos/peritomoreno.jpg', 
               'Imagery from International Space Station, courtesy Earth Science and Remote Sensing Unit, NASA Johnson Space Center',
               '', '',
               '400', '272',
              'https://eol.jsc.nasa.gov/SearchPhotos/photo.pl?mission=ISS016&roll=E&frame=24897'),
              
              ('RGI60-18.02342', 'Tasman Glacier', 
               'https://upload.wikimedia.org/wikipedia/commons/8/87/Lower_Tasman_Glacier_towards_Minarets.jpg', 
               '', 
               '(released into public domain by holder of the work)', '',
               '400', '266',
              'https://commons.wikimedia.org/wiki/File:Lower_Tasman_Glacier_towards_Minarets.jpg '),
              
            #  ('RGI60-19.00386', 'Ross Island', '', '', ''),
              ('RGI60-19.00595', 'Bahia Del Diablo Glacier', 
               'https://raw.githubusercontent.com/zschirmeister/glacier-gallery/dev3/glacier_photos/bahiadeldiablo.bmp', 
               'Photo courtesy P. Skvarca/National Snow and Ice Data Center', 
               '', '',
               '400', '260',
              'http://nsidc.org/data/glacier_photo/search/image_info/bahiadeldiablo20040224')
             ]
glacier_list = []
glacier_names = []
glacier_pics =[]
glpic_cite = []
glpic_lic = []
glpic_wi = []
glpic_he = []
glpic_src = []
for gl_no in Glacier_No:
    glacier_list.append(gl_no[0])
    glacier_names.append(gl_no[1])
    glacier_pics.append(gl_no[2])
    glpic_cite.append(gl_no[3])
    glpic_lic. append (gl_no[4])
    glpic_wi.append(gl_no[6])
    glpic_he.append(gl_no[7])
    glpic_src.append(gl_no[8])
    
df = df_all.loc[df_all['rgi_id'].isin(glacier_list)].copy()
df['name'] = glacier_names
df['Coords'] = list(zip(round(df.CenLat,2), round(df.CenLon,2)))
df

## Add glacier types and texts about the types

In [ ]:
# add glacier type 
df.loc[:, 'type'] = 'Pure ice glacier'
df.loc[df['name']== 'Tasman Glacier', 'type'] = 'Debris covered glacier'
df.loc[df['name']== 'Columbia Glacier', 'type'] = 'Calving glacier' # also debris covered
df.loc[df['name']== 'Perito Moreno', 'type'] = 'Calving glacier'
df.loc[df['name']== 'Mittivakkat Glacier', 'type'] = 'Ice cap'

# add glacier type description, #### Einzug wichtig!!!
glacier_text= """
    Glaciers are defined as a perennial 
    mass of land ice that froms through 
    the compression of snow. The ice has 
    to exhibit a past or present flow 
    with internal deformation or sliding
    at the base. They build up when 
    accumulation of snow exceeds 
    ablation. These two processes are
    balanced: accumulation in high 
    altitudes and ablation at low 
    altitudes and/or discharge into 
    the sea or lakes.

    (see IPCC, 2013: Climate Change 
    2013: The Physical Science Basis. 
    Contribution of Working Group I to 
    the Fifth Assessment Report of the
    Intergovern-mental Panel on Climate 
    Change [Stocker, T.F., D. Qin, 
    G.-K. Plattner, M. Tignor, S.K. Allen, 
    J. Boschung, A. Nauels, Y. Xia, V. Bex 
    and P.M. Midgley (eds.)]. Cambridge 
    University Press, Cambridge, United 
    Kingdom and New York, NY, USA, 
    1535 pp.)
"""
calving_glacier_text = '''  
    A glacier calves when a huge mass of
    ice breaks off at the front of the 
    glacier. Most of these glaciers are 
    water-terminating, e.g. in lakes, 
    fjords or the sea. They can float on 
    the surface or be grounded. But there 
    are also some glaciers which show 
    this phenomenon without terminating 
    in water. Calving glaciers exhibit many 
    crevasses due to their property of 
    fast flowing and a extensional 
    (stretching) flow at their lower end. 
    Another feature is their near-vertical 
    ice cliff where the process of calving 
    happens. Depending on the glacier 
    thickness the terminus can be between 
    tens to 80 m thick. How often and how 
    much a glacier calves is discribed by 
    the calving rate, which depends on the
    variations in the longitudinal strain 
    rates and the ice velocities. 
    
    (see Bishop, M. P., Björnsson, H., 
    Haeberli, W., Oerlemans, J., Shroder, J. F., 
    & Tranter, M. (2011). Encyclopedia of snow, 
    ice and glaciers. Springer Science & 
    Business Media.)'
    '''
debris_glacier_text = '''
    A layer of debris lies on the surface of 
    these glaciers, but only in the lower 
    part, so that the ice is covered. They 
    can be found in all major mountain 
    regions of the world. In the Himalayan 
    it is the predominant form of glaciation.
    The layer of debris forms by rockfall 
    from nearby mountain faces with high 
    denudation rates, rock avalanching and
    mixed snow/ice/rock avalanching. The 
    resulting distribution and type of the 
    debris layer influences the glacier’s 
    dynamics, rheology and mass balance, 
    especially its ablation. Depending on 
    the thickness of the layer, there is 
    an increase in melting or a decrease. 
    Dispersed debris cover and layers 
    thinner than ~15-80 mm enhace melting, 
    because they heat up under solar 
    radiation, whereas thicker layers 
    exhibit an insulating effect besides 
    preventing the ice from insolation.
    
    (see Bishop, M. P., Björnsson, H., 
    Haeberli, W., Oerlemans, J., Shroder, J. F., 
    & Tranter, M. (2011). Encyclopedia of snow, 
    ice and glaciers. Springer Science &
    Business Media.)'
    ''' 
ice_cap_text = '''
    Ice caps are defined as huge ice masses 
    that cover land on a large scale, but 
    do not extent 50000 km² (approximately 
    the area of Costa Rica). Therefore 
    Greenland and Antartica ice sheets are 
    excluded from the definition.
    Ice caps can be found in polar or 
    subtemperate regions, where accumulation 
    commonly exceeds ablation. As a result 
    glaciers grow in the higher parts of the 
    mountains, they merge and build ice bands 
    around the mountains which can fuse 
    glaciers from other mountains, building 
    the ice caps that cover large parts of the 
    land. Furthermore, they almost submerge 
    the topography beneath while only some 
    peaks could stick out of the ice, which 
    are colled “nunataks”. 
    
    (see Bishop, M. P., Björnsson, H., 
    Haeberli, W., Oerlemans, J., Shroder, 
    J. F., & Tranter, M. (2011). Encyclopedia 
    of snow, ice and glaciers. Springer 
    Science & Business Media.)
    '''

ice_glacier_text = ' Beautiful white blue shimmering glaciers...'


df.loc[df['type'] == 'Pure ice glacier', 'glaciertype_text'] = ice_glacier_text
df.loc[df['type'] == 'Calving glacier', 'glaciertype_text'] = calving_glacier_text
df.loc[df['type'] == 'Debris covered glacier', 'glaciertype_text'] = debris_glacier_text
df.loc[df['type'] == 'Ice cap', 'glaciertype_text'] = ice_cap_text

# define colors for dots of different glacier types 
df.loc[df['type'] == 'Pure ice glacier', 'color'] = '#85a9c1'
df.loc[df['type'] == 'Calving glacier', 'color'] = '#cc514e'
df.loc[df['type'] == 'Debris covered glacier', 'color'] = 'orange'
df.loc[df['type'] == 'Ice cap', 'color'] = 'white'

## Build special datastructure about all information

In [ ]:
# prepare coordinates  
df.loc[:, 'coords_x'] = df['Coords'].apply(lambda x: merc(x)[0])
df.loc[:, 'coords_y'] = df['Coords'].apply(lambda x: merc(x)[1])

# build sources
source = ColumnDataSource(
    data=dict(
                        x=list(df['coords_x']), 
                        y=list(df['coords_y']),
                        Area=list(df['Area']),
                        lon=list(df['CenLon']), 
                        lat=list(df['CenLat']),
                        img=glacier_pics,
                        gltype=list(df['type']),
                        gltype_text=list(df['glaciertype_text']),
                        empty1=[''] * len(df),
                        empty2=[''] * len(df),
                        name=list(df['name']),
                        colors=list(df['color']),
                        RGI_id=list(df['rgi_id']),
                        terminus=list(df['TerminusType']),
                        length=list(df['Lmax']),
                        pic_cite = glpic_cite,
                        pic_lic = glpic_lic,
                        pic_width=glpic_wi,
                        pic_height=glpic_he,
                        pic_src=glpic_src)
                       )

## Design for map with hover and webpage

In [ ]:
# Hover
TOOLTIPS = """
    <div>
        <div>
            <img
                src="@img" 
                height="@pic_height" 
                alt="image loading..." 
                width="@pic_width"
                style=" margin: 2px 2px 0px 2px;"
                border="2">
            </img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@name             </span> <br>
            <span style="font-size: 15px;                   ">@gltype           </span> <br>
            <span style="font-size: 15px;                   ">Length:           </span>
            <span style="font-size: 15px;                   ">@length           </span>
            <span style="font-size: 15px;                   ">m                 </span> <br>
            <span style="font-size: 15px;                   ">@terminus         </span> <br>
            <span style="font-size: 15px;                   ">Location          </span>
            <span style="font-size: 10px; color: #696;      ">@lat, @lon        </span> <br>
            <span style="font-size: 9px;                    ">@pic_cite @pic_lic</span>
        </div>
    </div>
"""
hover = HoverTool()
hover.tooltips = TOOLTIPS

############################################################################
p = figure(x_axis_type="mercator", 
           y_axis_type="mercator",
           plot_width = 1000,
           plot_height = 650,
           background='black',
           tools=[hover, 'tap', 'wheel_zoom', 'reset', 'pan'])

############################################################################
# Design figure
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False
p.xgrid.grid_line_color = None

############################################################################
# add map
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)
#p.min_border_top = 80
# add glacier dots
p.circle(x='x',
         y='y', 
         source=source,
         size=10,
         color='colors',
         line_color="black", 
         #fill_color="black",
         fill_alpha=0.9,
         name='needshover0',
            legend='gltype'
        )

############################################################################
# build a text box with information about glacier types
def construct_text_box(source): 
    # Plot and axes                                                             
    xdr = Range1d(0, 1220)                                                       
    ydr = Range1d(0, 1120)                                                       

    plot = Plot(                                        
        x_range=xdr, y_range=ydr,                                   
        plot_height=650, min_border=0, 
        plot_width=350
        )                                                                           
    # Add the writing 
    # textstyle:    https://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#fill-properties
    text = Text(x=10, y=5, text='empty1', text_color="#326a86", text_alpha=0.2, text_font='helvetica', text_font_size='10pt')# angle, angle_units, js_event_callbacks, js_property_callbacks, name, subscribed_events, 
                    #tags, text_align, text_baseline, text_color, text_font, text_font_style, text_line_height, x, x_offset, y, y_offset)
    title = Text(x=10, y=1055, text='empty2', text_color="#85a9c1", text_alpha=0.2, text_font='helvetica', text_font_size='14pt')
    plot.add_glyph(source, text)
    plot.add_glyph(source, title)    

    return plot

JS_CODE ="""
    // Get index of current selected datapoint
    sel_point_index = cb_data.source.attributes.selected["1d"]["indices"][0];     

    /* replace all name_display values with the name_label
       value of currently selected point */
    for (i=0; i < cb_data.source.data.gltype.length; i++) {
        cb_data.source.data.empty2[i] = cb_data.source.data.gltype[sel_point_index];
        }       
    cb_data.source.change.emit();
    
    for (i=0; i < cb_data.source.data.gltype.length; i++) {
        cb_data.source.data.empty1[i] = cb_data.source.data.gltype_text[sel_point_index];
        }      
     cb_data.source.change.emit();

    """

newtaptool = p.select(type=TapTool)
newtaptool.callback = CustomJS(code=JS_CODE)
text_box = construct_text_box(source)

############################################################################
# Design Application
# Panel
title       = '<div style="font-size:38px; color: #326a86; font-weight: bold" >Glacier Gallery</div>'
instruction = '<div style="font-size:15px; color: #326a86" >Explore glaciers of the world by moving the mouse over the colored dots on the map. When you click on them information about the glacier type will appear on the right.</div>'
oggm_logo   = '<a href="http://edu.oggm.org"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/oggm_loupe.png" width=220></a>'
pn_logo     = '<a href="https://panel.pyviz.org"><img src="http://panel.pyviz.org/_static/logo_stacked.png" width=60></a>'
links_to_license = '<div>Links to licenses of the images:</div>'
link1 = '<a href="https://creativecommons.org/licenses/by-sa/3.0/igo/deed.en">CC BY-SA 3.0 IGO </a>'
link2 = '<a href="https://creativecommons.org/licenses/by-nc/4.0/"> CC BY-NC</a>'
link3 = '<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC BY-SA 3.0 </a>'
link4 = '<a href="https://creativecommons.org/licenses/by/3.0/deed.en">CC BY 3.0 (Creative Commons Attribution 3.0 Unported)</a>'
links_pic_src = """<div>
                        Links to sources of the images:
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/lemoncreek20070917" >Lemon Creek Glacier</a>
                        <a href="https://commons.wikimedia.org/wiki/File:Columbia_Glacier_(Alaska)_by_Sentinel-2.jpg" >Columbia Glacier</a>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/white2012059902?order=true" >White Glacier</a>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/mittivakkat20130827?order=true" > Mittivakkat Glacier</a>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/briksdalsbreen20090730?order=true" >Briksdalsbreen</a>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/hintereisferner20060912?order=true" >Hintereisferner</a>
                        <a href="https://commons.wikimedia.org/wiki/File:Grosser_Aletschgletscher_3196.JPG" >Großer Aletschgletscher</a>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/tbilisa20109999" >Tbilisa Glacier</a>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/lewis20120223" >Lewis Glacier</a>
                        <a href="https://commons.wikimedia.org/wiki/File:Artesonraju_Peru.jpg" >Artesonraju</a>
                        <a href="https://eol.jsc.nasa.gov/SearchPhotos/photo.pl?mission=ISS016&roll=E&frame=24897" >Perito Moreno</a>
                        <a href="https://commons.wikimedia.org/wiki/File:Lower_Tasman_Glacier_towards_Minarets.jpg" >Tasman Glacier</a>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/bahiadeldiablo20040224" >Bahia Del Diablo Glacier</a>
                   </div>"""



header = pn.Row(pn.Pane(oggm_logo),  pn.layout.Spacer(width=30), 
                pn.Column(pn.Row(pn.Spacer(width=130), pn.Pane(title, width=400)), pn.Pane(instruction, width=1200)),
                pn.layout.HSpacer()
                )
link_list = pn.Row(link1, link2, link3, link4)

# Put plot and text together and show it
pn.Column(header, pn.Row(pn.Pane(pn_logo, width=100), p, pn.Spacer(width=5), text_box), links_to_license, link_list, links_pic_src, width_policy='max', height_policy='max').show()
#servable()